In [1]:
!pip install ipywidgets
!pip install deepface
!pip install ultralytics
!pip install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.2 MB/s eta 0:00:00:00:01
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114248 sha256=50350760f0ef46441b8f487450bca6371b4113af2e86b5cf37c74e464b3b9df7
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.3/886.3 kB 19.8 MB/s eta 0:00:0000:01
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-i41kzyzq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-i41kzyzq
  Resolved https://github.com/huggingface/transformers to comm

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob
import os
import imageio
import cv2
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from deepface import DeepFace
from ultralytics import YOLO
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import requests
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
import string
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Normalization
import pickle

24-11-15 08:32:24 - Directory /root/.deepface has been created
24-11-15 08:32:24 - Directory /root/.deepface/weights has been created
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
model = None
processor = None
is_qwen_model_loaded = False

def load_Qwen_model():
    global model, processor, is_qwen_model_loaded
    if not is_qwen_model_loaded:
        # Load the Qwen model and processor
        model = Qwen2VLForConditionalGeneration.from_pretrained(
            "Qwen/Qwen2-VL-7B-Instruct",
            torch_dtype="auto",
            device_map="auto",
        )

        processor = AutoProcessor.from_pretrained(
            "Qwen/Qwen2-VL-7B-Instruct"
        )
        is_qwen_model_loaded = True
        print("Qwen model loaded.")

In [4]:
def build_prompt(img_id, data_csv):
    # Step 1: Ensure `img_id` is a string
    img_id = str(img_id)
    
    # Step 2: Remove leading zeros to match the image ID format in the CSV
    img_name = img_id.lstrip('0')  # Removes leading zeros
    
    # Step 3: Find the row corresponding to `img_name`
    result = data_csv[data_csv['id'] == int(img_name)]  # Assuming 'id' is the column with image IDs

    # If no matching record is found
    if result.empty:
        return "No relevant information available."

    # Step 4: Extract the first matching row (assuming unique `id` values)
    row = result.iloc[0]

    # Step 5: Build the context
    context_parts = []
    
    if pd.notna(row['detected_objects']):
        context_parts.append(f"Detected objects: {row['detected_objects']}")
    
    if pd.notna(row['Dominant Emotion']):
        context_parts.append(f"Dominant emotion: {row['Dominant Emotion']}")
    
    if pd.notna(row['Dominant Race']):
        context_parts.append(f"Race: {row['Dominant Race']}")
    
    if pd.notna(row['gender']):
        context_parts.append(f"Gender: {row['gender']}")
    
    if pd.notna(row['age_group']):
        context_parts.append(f"Age group: {row['age_group']}")
    
    if pd.notna(row['text']):
        context_parts.append(f"Text in meme: '{row['text']}'")
    
    if pd.notna(row['sentiment_analysis']):
        context_parts.append(f"Sentiment: {row['sentiment_analysis']}")

    # Join context parts with ". " and end with a period
    prompt = ". ".join(context_parts) + "." if context_parts else "No relevant information available."
    return prompt

In [5]:
# Passing necessary datasets
train_file = '/kaggle/input/facebook-hateful-meme-dataset/data/train.jsonl'
img_folder = '/kaggle/input/facebook-hateful-meme-dataset/data/img'  
output_csv_file = '/kaggle/working/results.csv'
data_path = '/kaggle/input/meme-final-datasets/final_datasets.csv'

In [7]:
data_csv = pd.read_csv(data_path)
img_id = '1235'
build_prompt(img_id, data_csv)

"Detected objects: person. Dominant emotion: happy. Race: middle eastern. Gender: man. Age group: 35-44 years. Text in meme: 'when you're feeling horny asf but your habibi is on periods let's try a goat'. Sentiment: bad."

In [14]:
def hateful_meme_detection(image_path, img_name, data_csv):
    global prompt, uploaded_image_path, prediction_result, reason, predicted_sentiments
    load_Qwen_model()
    
    prompt = build_prompt(img_name, data_csv)
    print(prompt)
    # Step 1: Load the single image
    global_uploaded_image = Image.open(image_path)

    # Step 2: Prepare the global prompt
    data = f"Please predict whether the content in the meme is hateful or non-hateful. Context: {prompt}"

    # Step 3: Prepare the messages for the model, including the global prompt
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                },
                {
                    "type": "text",
                    "text": data,
                },
            ],
        }
    ]

    # Step 4: Apply the chat template and create inputs for the model
    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(
        text=[text_prompt],
        images=[global_uploaded_image],
        padding=True,
        return_tensors="pt",
    )

    inputs = inputs.to("cuda")

    # Step 5: Generate the output from the model
    output_ids = model.generate(**inputs, max_new_tokens=1024)
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    # Step 6: Replace '\n' with a space in the output text
    output_text = [text.replace('\n', ' ') for text in output_text]

    # Step 7: Store the final output in global variables
    reason = ' '.join(output_text)

    # Determine prediction based on the final output
    if "non-hateful" in reason.lower():
        prediction_result = 0  # Non-hateful
    else:
        prediction_result = 1  # Hateful

    # Step 8: Print the results
    print(f"Prediction: {'Non-Hateful' if prediction_result == 0 else 'Hateful'},\nReason: {reason}\n")


In [9]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [15]:
import json
import os
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

# Load data from the given JSONL file
def load_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Save predictions to a CSV file
def save_predictions_to_csv(predictions, output_file):
    df = pd.DataFrame(predictions, columns=['id', 'actual_label', 'predicted_label', 'result', 'reason'])
    df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

# Analyze predictions and generate classification metrics
def analyze_results(predictions):
    actual_labels = [entry['actual_label'] for entry in predictions]
    predicted_labels = [entry['predicted_label'] for entry in predictions]

    # Classification report
    report = classification_report(actual_labels, predicted_labels)
    print("Classification Report:\n", report)

    # Accuracy
    accuracy = accuracy_score(actual_labels, predicted_labels)
    print(f"Accuracy: {accuracy:.2f}")
    
    # Precision
    precision = precision_score(actual_labels, predicted_labels, pos_label=1)
    print(f"Precision: {precision:.2f}")

    # Recall
    recall = recall_score(actual_labels, predicted_labels, pos_label=1)
    print(f"Recall: {recall:.2f}")

    # Confusion Matrix
    cm = confusion_matrix(actual_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Hateful', 'Hateful'], yticklabels=['Non-Hateful', 'Hateful'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

In [18]:
# Main function predict and analyse
def process_memes_and_predict(file_path, img_folder, output_csv, data_path):
    # Step 1: Load data from the JSONL file
    data = load_data(file_path)
    data_csv = pd.read_csv(data_path)

    # Step 2: Storing the result
    predictions = []
    
    # Start time
    start_time = time.time()
    
    # Define your range here (Avi running for only 5 images)
    for entry in tqdm(data[:], desc="Processing Memes", unit="image"):
        img_id = entry['id']
        actual_label = entry['label']

        # Step 3: Finding corresponding image file by removing leading zeros from the image ID
        img_name = str(img_id).zfill(5) + ".png"
        img_path = os.path.join(img_folder, img_name)

        img_csv = str(img_id).zfill(5)
        # Step 4: Predict using hateful_meme_detection function
        hateful_meme_detection(img_path, img_csv, data_csv) 
        # Storing the result        
        predicted_label = prediction_result
        xx = "hateful" if predicted_label == 1 else "non-hateful"

        # Storing the results
        prediction_entry = {
            'id': img_id,
            'actual_label': actual_label,  # Store the actual label from the dataset
            'predicted_label': predicted_label,
            'result': xx,
            'reason': reason
        }
        predictions.append(prediction_entry)
    
    # End Time    
    elapsed_time = time.time() - start_time 
    print(f"Processing completed in {elapsed_time:.2f} seconds.")

    # Step 5: Save results to CSV
    save_predictions_to_csv(predictions, output_csv)

    # Step 6: Analyze results
    analyze_results(predictions)

In [ ]:
process_memes_and_predict(train_file, img_folder, output_csv_file, data_path)

Processing Memes:   0%|          | 0/8500 [00:00<?, ?image/s]

Detected objects: person, person, tie. Text in meme: 'The text in the image says:  "its their character not their color that matters"'. Sentiment: bad.


Processing Memes:   0%|          | 1/8500 [00:02<6:17:14,  2.66s/image]

Prediction: Non-Hateful,
Reason: The content in the meme is non-hateful.

Detected objects: person, person, tie. Text in meme: 'don't be afraid to love again everyone is not like your ex'. Sentiment: good.


Processing Memes:   0%|          | 2/8500 [00:09<11:57:53,  5.07s/image]

...

Prediction: Hateful,
Reason: The content in the meme is hateful. The text "what about the irish slaves??" is a reference to a historical period when Irish people were also enslaved, which is a sensitive and controversial topic. The use of this phrase in a meme is likely intended to be provocative and offensive, contributing to a negative sentiment.

Detected objects: person, person, tie, tie. Dominant emotion: sad. Race: white. Gender: man. Age group: 35-44 years. Text in meme: 'yeah bitches totalinarism is a sham'. Sentiment: bad.


Processing Memes:  12%|█▏        | 982/8500 [1:48:43<11:01:18,  5.28s/image]

Prediction: Hateful,
Reason: The content in the meme is hateful.

Detected objects: person, truck. Dominant emotion: neutral. Race: white. Gender: man. Age group: 35-44 years. Text in meme: 'ready for a king of the hill marathon?'. Sentiment: bad.


Processing Memes:  12%|█▏        | 983/8500 [1:48:47<10:10:29,  4.87s/image]

Prediction: Non-Hateful,
Reason: The content in the meme is non-hateful.

Text in meme: 'you rape em we scrape em no fetus can defetus'. Sentiment: neutral.
